# ¿Cuál es el estado con mejor rating promedio en sus negocios? ¿Por qué?

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
sqlContext = SQLContext(sc)
df = pd.read_csv('/content/drive/MyDrive/ODD tp 1 dataset/business.csv')
df = df.to_parquet('/content/drive/MyDrive/ODD tp 1 dataset/business2.parquet')

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
df = sqlContext.read.parquet('/content/drive/MyDrive/ODD tp 1 dataset/business2.parquet', header=True, inferSchema=True)
rdd = df.rdd

In [ ]:
rdd.take(5)

[Row(business_id='Pns2l4eNsfO8kk83dixA6A', name='Abby Rappoport, LAC, CMQ', address='1616 Chapala St, Ste 2', city='Santa Barbara', state='CA', postal_code='93101', latitude=34.4266787, longitude=-119.7111968, stars=5.0, review_count=0.0, is_open=0, attributes="{'ByAppointmentOnly': 'True'}", categories='Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists', hours=None),
 Row(business_id='mpf3x-BjTdTEA3yCZrAYPw', name='The UPS Store', address='87 Grasso Plaza Shopping Center', city='Affton', state='MO', postal_code='63123', latitude=38.551126, longitude=-90.335695, stars=3.0, review_count=8.0, is_open=1, attributes="{'BusinessAcceptsCreditCards': 'True'}", categories='Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services', hours="{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}"),
 Row(business_id='tUFrWirKiKi_TAns

In [ ]:
rdd.map(lambda x: x.stars).distinct().collect()

[4.0, 4.5, 2.0, 2.5, 5.0, 3.0, 3.5, 1.5, 1.0]

In [ ]:
rdd_sliced = rdd.map(lambda x: (x.state, x.stars))

In [ ]:
# hago que las stars sean key y pongo un 1 para contar. 
# sumo las stars y sumo las ocurrencias.  
# para sacar el promedio divido entre la suma de los puntajes (stars) y la cantidad de puntajes que habia


estado_mejor_promedio = rdd_sliced \
    .mapValues(lambda v: (v, 1)) \
    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
    .mapValues(lambda v: v[0]/v[1]) \
    .takeOrdered(1, lambda x: -x[1])

## El Estado con mejor rating promedio en sus negocios es...

In [ ]:
estado_mejor_promedio

[('MT', 5.0)]

## ¿Por qué? Porque tiene solo una review para sus negocios. Los Estados con más reviews tienen menor probabilidad de tener un rating promedio tan alto debido a que la muestra es mayor. 



In [ ]:
rdd_sliced.map(lambda x: (x[0], 1)).countByKey()

defaultdict(int,
            {'CA': 5203,
             'MO': 10913,
             'AZ': 9912,
             'PA': 34039,
             'TN': 12056,
             'FL': 26330,
             'IN': 11247,
             'LA': 9924,
             'AB': 5573,
             'NV': 7715,
             'ID': 4467,
             'DE': 2265,
             'IL': 2145,
             'NJ': 8536,
             'NC': 1,
             'CO': 3,
             'WA': 2,
             'HI': 2,
             'UT': 1,
             'TX': 4,
             'MT': 1,
             'MI': 1,
             'SD': 1,
             'XMS': 1,
             'MA': 2,
             'VI': 1,
             'VT': 1})